Pierwszym krokiem jest przygotowanie zdjęć do analizy.

Importy:

In [1]:
from PIL import Image
import os
import keras
import sklearn

import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.inception_v3 import InceptionV3


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Wybranie ścieżek oraz docelowego rozmiaru zdjęć kotów.

In [2]:
breeds_folder = '/kaggle/input/cats-breed-dataset/cat_v1'
output_folder = '/kaggle/working/train'

target_size = (224, 224)

Następnie iterujemy po folderach i dokonujemy zmian w rozmiarze zdjęcia. Tworzymy także podfoldery raz w folderze wyjściowym. Z powodu istnienia plików o nieprawidłowym rozszerzeniu, musimy też je odfiltrować.

In [3]:
all_images = []
all_labels = []

breeds_folder = '/kaggle/input/cats-breed-dataset/cat_v1'

for breed_folder in os.listdir(breeds_folder):
    breed_images = os.listdir(os.path.join(breeds_folder, breed_folder))
    all_images.extend([os.path.join(breeds_folder, breed_folder, img) for img in breed_images])
    all_labels.extend([breed_folder]*len(breed_images))

df = pd.DataFrame({'filename': all_images, 'class': all_labels})

In [4]:
valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')
for x in df.filename:
    if not x.lower().endswith(valid_extensions):
        df = df.drop(df[df.filename == x].index)
df = df.sample(frac=1)
df = df.reset_index(drop=True)
df.head(10)

,filename,class
0,/kaggle/input/cats-breed-dataset/cat_v1/maine_...,maine_coon
1,/kaggle/input/cats-breed-dataset/cat_v1/bengal...,bengal
2,/kaggle/input/cats-breed-dataset/cat_v1/ragdol...,ragdoll
3,/kaggle/input/cats-breed-dataset/cat_v1/ragdol...,ragdoll
4,/kaggle/input/cats-breed-dataset/cat_v1/bengal...,bengal
5,/kaggle/input/cats-breed-dataset/cat_v1/domest...,domestic_shorthair
6,/kaggle/input/cats-breed-dataset/cat_v1/ragdol...,ragdoll
7,/kaggle/input/cats-breed-dataset/cat_v1/bengal...,bengal
8,/kaggle/input/cats-breed-dataset/cat_v1/ragdol...,ragdoll
9,/kaggle/input/cats-breed-dataset/cat_v1/maine_...,maine_coon


In [5]:
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['class'], random_state=42)

input_shape = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(train_df, 
                                                    x_col='filename', 
                                                    y_col='class', 
                                                    target_size=target_size, 
                                                    batch_size=batch_size, 
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                    shuffle=True)
val_generator = val_datagen.flow_from_dataframe(val_df, 
                                                x_col='filename', 
                                                y_col='class', 
                                                target_size=target_size, 
                                                batch_size=batch_size, 
                                                color_mode='rgb',
                                                class_mode='categorical',
                                                shuffle=False)
num_classes = train_df['class'].nunique()

Found 760 validated image filenames belonging to 5 classes.
Found 191 validated image filenames belonging to 5 classes.


In [6]:
keys = list(train_generator.class_indices.keys())
values = list(train_generator.class_indices.values())
train_generator.class_indices

{'bengal': 0,
 'domestic_shorthair': 1,
 'maine_coon': 2,
 'ragdoll': 3,
 'siamese': 4}

In [7]:
df = pd.DataFrame({'Breed':keys,
                'Index':values})
df.to_csv('indexes.csv', encoding='utf-8', index=False)
df.head()

,Breed,Index
0,bengal,0
1,domestic_shorthair,1
2,maine_coon,2
3,ragdoll,3
4,siamese,4


Następnie przechodzimy do przygotowania zdjęć w folderze wyjściowym.

Tworzenie modelu

In [8]:
baseModel = DenseNet121(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(num_classes, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in model.layers[:-8]:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 10

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    batch_size=batch_size
)

29084464/29084464 [==============================] - 0s 0us/step
Epoch 1/10
23/23 [==============================] - 68s 2s/step - loss: 19.8119 - accuracy: 0.4519 - val_loss: 2.4346 - val_accuracy: 0.7625
Epoch 2/10
23/23 [==============================] - 44s 2s/step - loss: 2.0619 - accuracy: 0.7734 - val_loss: 0.9716 - val_accuracy: 0.7812
Epoch 3/10
23/23 [==============================] - 44s 2s/step - loss: 0.5111 - accuracy: 0.8407 - val_loss: 0.6521 - val_accuracy: 0.8062
Epoch 4/10
23/23 [==============================] - 43s 2s/step - loss: 0.3817 - accuracy: 0.8805 - val_loss: 0.5068 - val_accuracy: 0.8562
Epoch 5/10
23/23 [==============================] - 41s 2s/step - loss: 0.3156 - accuracy: 0.8777 - val_loss: 0.5438 - val_accuracy: 0.8375
Epoch 6/10
23/23 [==============================] - 42s 2s/step - loss: 0.2757 - accuracy: 0.8984 - val_loss: 0.5560 - val_accuracy: 0.8375
Epoch 7/10
23/23 [==============================] - 43s 2s/step - loss: 0.2467 - accuracy: 0.9

In [9]:
model.save('models/densenet121.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
baseModel = ResNet101(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(num_classes, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in model.layers[:-8]:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)

171446536/171446536 [==============================] - 1s 0us/step
Epoch 1/10
23/23 [==============================] - 54s 2s/step - loss: 8.6346 - accuracy: 0.2679 - val_loss: 2.0056 - val_accuracy: 0.1937
Epoch 2/10
23/23 [==============================] - 42s 2s/step - loss: 1.5302 - accuracy: 0.2962 - val_loss: 1.7051 - val_accuracy: 0.1875
Epoch 3/10
23/23 [==============================] - 42s 2s/step - loss: 1.5115 - accuracy: 0.3008 - val_loss: 1.4970 - val_accuracy: 0.3125
Epoch 4/10
23/23 [==============================] - 40s 2s/step - loss: 1.4762 - accuracy: 0.3255 - val_loss: 1.4920 - val_accuracy: 0.2750
Epoch 5/10
23/23 [==============================] - 41s 2s/step - loss: 1.4529 - accuracy: 0.3420 - val_loss: 1.5588 - val_accuracy: 0.2875
Epoch 6/10
23/23 [==============================] - 41s 2s/step - loss: 1.4703 - accuracy: 0.3613 - val_loss: 1.5080 - val_accuracy: 0.3688
Epoch 7/10
23/23 [==============================] - 43s 2s/step - loss: 1.4140 - accuracy: 0.

In [11]:
model.save('models/resnet101.h5')

In [12]:
baseModel = InceptionV3(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

input_shape = (224, 224)

headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(num_classes, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in model.layers[:-8]:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)

87910968/87910968 [==============================] - 0s 0us/step
Epoch 1/10
23/23 [==============================] - 50s 2s/step - loss: 15.3957 - accuracy: 0.5755 - val_loss: 5.5368 - val_accuracy: 0.7312
Epoch 2/10
23/23 [==============================] - 39s 2s/step - loss: 4.2819 - accuracy: 0.7541 - val_loss: 1.8578 - val_accuracy: 0.7937
Epoch 3/10
23/23 [==============================] - 42s 2s/step - loss: 1.1636 - accuracy: 0.8365 - val_loss: 0.8464 - val_accuracy: 0.8125
Epoch 4/10
23/23 [==============================] - 43s 2s/step - loss: 0.5072 - accuracy: 0.8585 - val_loss: 0.5885 - val_accuracy: 0.8375
Epoch 5/10
23/23 [==============================] - 43s 2s/step - loss: 0.2846 - accuracy: 0.9176 - val_loss: 0.5841 - val_accuracy: 0.8313
Epoch 6/10
23/23 [==============================] - 42s 2s/step - loss: 0.2265 - accuracy: 0.9231 - val_loss: 0.6696 - val_accuracy: 0.8313
Epoch 7/10
23/23 [==============================] - 41s 2s/step - loss: 0.2002 - accuracy: 0.9

In [13]:
model.save('models/inceptionV3.h5')